This document is created to remember and have some stuff about Pillow Library to copy paste in the future

In [1]:
# General variables 
DATA_PATH = './../../../data/pillow_data/'
IMG_PATH = './../../../data/cat.jpg'

THUMBNAIL_SIZE = (64, 64)


In [2]:
from PIL import Image

# Load an Image from scraptch and obtain its full info
img = Image.open(IMG_PATH)      # Image reading from the disk

print(
    img.format,                 # Image format (JPG, PNG, PPT...)
    img.size,                   # Image Size XY
    img.mode                    # Image mode, RGB or black & whites
)

img.show()

JPEG (340, 340) RGB


[1] Creating a PNG thumbnails

In [37]:
img = Image.open(IMG_PATH)

# Trying to convert the image 
img.thumbnail(THUMBNAIL_SIZE)
img.save(DATA_PATH+'new_thumbnail.png', 'PNG')

[2] Cropping the image region and doing some stuff<br>
(a,b,c,d) -> space from (left, top, right, bottom)

In [69]:
img = Image.open(IMG_PATH)
print(img.size)
crop_box = (20,20,320,320)

region = img.crop(crop_box)
region = region.transpose(Image.Transpose.ROTATE_90)       # Rotating rhe image 90deg
img.paste(region,crop_box)                                  # Pasting the aimage again in its old position but changed
# img.show()
img.save(DATA_PATH+'transposed_cat.png', 'PNG')

(340, 340)


[3] Rolling the image

In [49]:
def roll(im, delta):
    """Roll an image sideways."""
    xsize, ysize = im.size

    delta = delta % xsize
    if delta == 0:
        return im

    part1 = im.crop((0, 0, delta, ysize))
    part2 = im.crop((delta, 0, xsize, ysize))
    im.paste(part1, (xsize - delta, 0, xsize, ysize))
    im.paste(part2, (0, 0, xsize - delta, ysize))

    return im

img = Image.open(IMG_PATH)
new_img = roll(img, img.size[0]//2)                     # Rolling the immage the half x dim, just in middle
new_img.show()

[4] Creating two images and merging them

In [72]:
crop_box = (40,40,300,300)
img = Image.open(IMG_PATH)
img2 = img.copy()

region = img.crop(crop_box)
region = region.transpose(Image.Transpose.ROTATE_90)
img.paste(region, crop_box)

new_width = img.size[0] + img2.size[0]                  # Obtaining the end x value
new_height = max(img.size[1], img2.size[1])             # Y value

new_img = Image.new('RGB', (new_width, new_height))    # Creating totally new img
new_img.paste(img)                                      # Pasting the image in 0x,y0
new_img.paste(img2, (img.size[0],0))                    # Pasting the img 2 at the end of the img and on top of the new Image

new_img.save(DATA_PATH+'two_images.jpeg', 'JPEG')
# new_img.show()

[5] You can split the image bands if you need to work with them

In [74]:
img = Image.open(IMG_PATH)
r, g, b = img.split()                                    # Obtaining the image splitted in their bands

b.save(DATA_PATH+'black_white.png', 'PNG')               # So, we obtain the black band

merged = Image.merge('RGB', (g,b,r))                     # Merging the image bands again but in different order

merged.save(DATA_PATH+'split_5.jpeg', 'JPEG')
merged.show()


[6] Geometry transforms and resizing

In [79]:
img = Image.open(IMG_PATH)

resized_img = img.resize((64, 64))                              # Resizing the image into a new value
print(f'Ond size: {img.size}, New size: {resized_img.size}')
resized_img.save(DATA_PATH+'resized_6.jpeg', 'jpeg')

rotated_img = img.rotate(60)                                    # Rotating the image 60deg in counter clockwise (with the base img as overflit:hidden)
# rotated_img.show()          

Ond size: (340, 340), New size: (64, 64)


[7] CSS into Python with Pillow, oh Damn!

In [83]:
from PIL import Image, ImageOps

img = Image.open(IMG_PATH)

size = (150, 300)

ImageOps.contain(img, size).save(DATA_PATH+'/like_css/contain.jpeg')                # Lke in CSS object-fit:contain 
ImageOps.cover(img, size).save(DATA_PATH+'/like_css/cover.jpeg')                    # By max value, so occupy all the space
ImageOps.fit(img, size).save(DATA_PATH+'/like_css/fit.jpeg')                        # Lke in CSS object-fit:cover
ImageOps.pad(img, size, color="#f2f2f2").save(DATA_PATH+'/like_css/pad.jpeg')       # CSS object-fit:contain but adds colored padding for white spaces


[8] Color transformations <br>

In [16]:
from PIL import Image

"""
The library supports transformations between each supported mode and the “L” and “RGB” modes. 
To convert between other modes, you may have to use an intermediate image (typically an “RGB” image).

    1 (1-bit pixels, black and white, stored with one pixel per byte)

    L (8-bit pixels, grayscale)

    P (8-bit pixels, mapped to any other mode using a color palette)

    RGB (3x8-bit pixels, true color)

    RGBA (4x8-bit pixels, true color with transparency mask)

    CMYK (4x8-bit pixels, color separation)

    YCbCr (3x8-bit pixels, color video format) Note that this refers to the JPEG, and not the ITU-R BT.2020, standard

    LAB (3x8-bit pixels, the L*a*b color space)

    HSV (3x8-bit pixels, Hue, Saturation, Value color space) Hue’s range of 0-255 is a scaled version of 0 degrees <= Hue < 360 degrees

    I (32-bit signed integer pixels)

    F (32-bit floating point pixels)
"""
with Image.open(IMG_PATH) as img:
    new_img = img.convert("L")          # Gray Scale (L = R * 299/1000 + G * 587/1000 + B * 114/1000)
    new_img = img.convert("1")

    # The following example converts an RGB image (linearly calibrated according to ITU-R 709, using the D65 luminant) to the CIE XYZ color space:
    rgb2xyz = (
        0.412453, 0.357580, 0.180423, 0,
        0.212671, 0.715160, 0.072169, 0,
        0.019334, 0.119193, 0.950227, 0
    )
    new_img = img.convert("RGB", rgb2xyz)
    new_img.show()

[7] Image Filters

In [3]:
from PIL import Image, ImageFilter

""" 
    Pillow provides some predefined filters:
    BLUR
    CONTOUR
    DETAIL
    EDGE_ENHANCE
    EDGE_ENHANCE_MORE
    EMBOSS
    FIND_EDGES
    SHARPEN
    SMOOTH
    SMOOTH_MORE
"""

img = Image.open(IMG_PATH)

""" 
The EMBOSS filter is used to create a three-dimensional effect in an image by highlighting edges and structures. 
It works by calculating differences between adjacent pixels in the image and adjusting pixel values based on these calculations.
"""
new_img = img.filter(ImageFilter.EMBOSS)
new_img.show()
new_img.save(DATA_PATH+'EMBOSS_7.png')

[8] Points operations

In [15]:
"""
The point() method can be used to translate the pixel values of an image (e.g. image contrast manipulation). 
In most cases, a function object expecting one argument can be passed to this method. Each pixel is processed according to that function:
"""
from PIL import Image 

img = Image.open(IMG_PATH)

# Applynf point transforms
new_img = img.point(lambda i: i*0.3) # Changing the image contrast
# new_img.show()


# Playing with the individual bandas of the image 
# Obtaining the image bands 
i_bands = img.split()

R,G,B = 0, 1, 2

# Selecting the regions with the RED Band color intensity is less than 100, in the pic is the white area
mask = i_bands[R].point(lambda i: i < 100 and 255)          # The and 255 in binary is 11111111, so we limit the range between 0 and 255 deleting possible errors

out = i_bands[G].point(lambda i: i*0.7)                     # Procdessing th egreen band

# Paste the processed band back, but only where red was < 100
i_bands[G].paste(out, None, mask) # Image.paste(im, box=None, mask=None)

i_bands[G].show()
